# Numerical integration with unequal intervals in python 

Numerical integration is python is straightforawrd using the `scipy` routines. However, most scipy routines uses equal spaced intervals to perform the integrals. If we have data stream in the (x,y) format where x is not equally spaced then we might have to write our own routines. These routines will be substantially slower than the ones that can be implemented in C++ but they integrate well if the rest of the analysis is done in pandas. Here, we will demonstrate how to perform numerical integration using trapezoid rule over intervals which are not equally spaced.

We begin with demonstrating definite integrals of our chosen function so that we know the exact numerical values to expect.  

Then we will perform the integral using scipy with equal-spaced intervals.  

At the end we will generate a unequally spaced stream of (x,y) from our function and perform numerical integration on the function using trapezoidal rule.


## Exact integration with Sagemath

In [1]:
from sage.symbolic.integration.integral import definite_integral
from sage.symbolic.integration.integral import indefinite_integral

We will integrate the function $y(x) = 2x - 3x^2 + 0.5*x^4$ over the interval (3, 5). 

In [11]:
x = var('x')
y = 2*x - 3*x*x + 0.5*x**4

In [12]:
y.integrate(x)

0.1*x^5 - x^3 + x^2

In [13]:
y.integrate(x, 3, 5)

206.2

## Numerical integration with scipy

Now that we know the exact answer, we will perform the same integration using `scipy`.   
Note that the Sagemath portion of the code is run using the Sagemath kernel. The rest of the notebook will use Python 3 kernel. Sagemath will not be able to handle the pandas 
since it uses rings of integers instead of integer datatypes. 

In [38]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate
from matplotlib import pyplot as plt
import pandas as pd

In [39]:
def f(x):
    y = 2*x - 3*x*x + 0.5*x**4
    return y

In [40]:
integrate.quad(lambda x: f(x), 3, 5)

(206.20000000000002, 2.2892798767770728e-12)

## Numerical integration with custom trapezoidal function using equal spacing

First we genearte equally spaced x values and then generate the (x, y) series.

In [46]:
x = np.arange(0, 10, 0.25) 
y = 2*x - 3*x*x + 0.5*x**4

In [47]:
def trapezoid(x, y, window=2, xstart=0, xend=10):
    if len(x) != len(y):
        return -1
    df = pd.DataFrame({'x':x, 'y':y})
    df = df[(df['x'] >= xstart) & (df['x'] <= xend)]
    df['tr'] = df['y'].rolling(window=window).mean()
    df['xdiff'] = df['x'].diff(periods=1)
    df['integration'] = df['tr']*df['xdiff']   
    df['integrationcumulative'] = df['integration'].cumsum()
    result = df['integrationcumulative'].iloc[-1]
    return result 

In [48]:
trapezoid(x,y, xstart=3, xend=5, window=2)

207.158203125

There is a slight error as out interval size is quite big. The error term can be exactly calulated but we will not discuss it here.

## Numerical integration with custom trapezoidal function using unequal spacing

In [49]:
x = np.sort(np.random.random(1000)*10)
y = 2*x - 3*x*x + 0.5*x**4

In [50]:
trapezoid(x,y, xstart=3, xend=5, window=2)

203.3448070434829

The error is larger but the integration routine is quite stable.